In [12]:
import spacy
import json
from spacy import displacy


tokenizer = spacy.load('en',disable=['parser','ner'])
parser = spacy.load('en',disable=['tagger'])
all = spacy.load('en')

In [17]:
def json_to_spacy(json_):
    json_ = json.loads(json_)
    text = json_['text']
    
    denotations = []
    # start, end, length, id
    for denotation in json_['denotations']:
        id_ = denotation['id']
        begin = denotation['span']['begin']
        end = denotation['span']['end']
        length = end - begin
        denotations.append({'id':id_,'begin':begin,'end':end,'length':length})
    
    denotations_begins = {}
    for denotation in denotations:
        if denotation['begin'] not in denotations_begins:
            denotations_begins[denotation['begin']] = []
        denotations_begins[denotation['begin']].append(denotation)
        
    longest_denotations = []
    current = -1
    for key, denotations in denotations_begins.items():
        if denotations[0]['begin'] > current:
            longest_denotation = max(denotations, key = lambda i: i['length'])
            longest_denotations.append(longest_denotation)
            current = longest_denotation['end']
        
    tokens = []
    tokens_ws = []
    current_denotation = 0
    advancement = 0
    endgame = False
    for token in nlp(text):
        if token.idx < advancement:
            continue
        
        if token.idx + len(token.text_with_ws) <= longest_denotations[current_denotation]['begin'] or endgame:
            tokens.append(token.text)
            tokens_ws.append(len(token.text) != len(token.text_with_ws))
            
        else:
            begin = longest_denotations[current_denotation]['begin']
            end = longest_denotations[current_denotation]['end']
            token_text = text[begin:end]
            tokens.append(token_text)
            tokens_ws.append(text[end:end+1] == ' ')
            
            advancement = end
                            
            if current_denotation + 1 < len(longest_denotations):
                current_denotation += 1
            else:
                endgame = True
                
    doc = spacy.tokens.Doc(nlp.vocab, words=tokens,
          spaces=tokens_ws)
    
    for name, proc in parser.pipeline:
        doc = proc(doc)
        
    return doc
    
    
f = open('short.json','r')
doc2 = json_to_spacy(f.read())
doc = all('Complex formation of platelet membrane glycoproteins IIb and IIIa with the fibrinogen D domain.')
displacy.render(doc, style='dep',jupyter=True)
svg1 = displacy.render(doc, style='dep',jupyter=False)
displacy.render(doc2, style='dep',jupyter=True)
svg2 = displacy.render(doc2, style='dep',jupyter=False)

open('parse_without_ner.svg','w', encoding='utf-8').write(svg1)
open('parse_with_ner.svg','w', encoding='utf-8').write(svg2)


8855